In [1]:
library(tidyverse) # metapackage with lots of helpful functions
library(data.table)
library(DT)
library(readxl)
library(dplyr)
library(data.table)
library(xts)
library(forecast)
library(lubridate)
library(data.table)
library(mltools)
library(tseries)
library(TTR)
library(stats)
library(reticulate)
library(data.table)
library(dplyr)
require(lubridate)
require(stringr)
require(Metrics)
require(zoo)
library(rhdf5)

path = '../input/ashrae-energy-prediction/'
ucf_root = '../input/ashrae-ucf-spider-and-eda-full-test-labels/'
ucl_root ='../input/ucl-data-leakage-episode-2/'
asu_root ='../input/asu-kernel-leach/'
cor_root ='../input/ashrae-site15-cornell/'

leak = FALSE
tolag = FALSE
to_diff = FALSE
to_log = FALSE

list.files(path = path)
logger = function(x){
    return(sign(x)*log(1+abs(x)))
}
differ = function(x,n=1){
    l = logger(x)
    return(l-shift(l,n))
}

relative_temp =function(TD,T){
    return(100*exp((17.625*TD)/(243.04+TD))/exp((17.625*T)/(243.04+T)))
}
farenhait_converter = function(T){
    return(T*1.8+32)
}
celsius_converter = function(T){
    return((T-32)/1.8)
}
heat_index = function(T,RH){
    T = farenhait_converter(T)
    HI = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH
    return(HI)#celsius_converter(HI))
}
trunc <- function(x, prec = 1,...) base::trunc(x * 10^prec, ...) / 10^prec;

weight_average = function(x){
    len=length(x)
    weight = seq(1,len)/(len*(len+1)/2)
    return(trunc(sum(x*weight)))
}
wind_converter <- function(x){
    return(2.23694*x)
}
wind_chill =function(v,t){
    t = farenhait_converter(t)
    v = wind_converter(v)
    return(ifelse(v>=3 & t<=50,35.74 + (0.6215 * t) - 35.75 * (v^0.16) + 0.4275 * t * v^0.16,NA))
}
feel_like = function(v,t,rh){
    tf = farenhait_converter(t)
    vf = wind_converter(v)
    res = tf
    if(length(which(tf<=50 & vf>=3))>0){
        res[which(tf<=50 & vf>=3)] <- wind_chill(v[which(tf<=50 & vf>=3)],t[which(tf<=50 & vf>=3)])
    }
    if(length(which(tf>=80))){
        res[which(tf>=80)] <- heat_index(t[which(tf>=80)],rh[which(tf>=80)])
    }
    return(res)
}
freq_na_calc <- function(x){
    na_num = sum(is.na(x))
    tab = table(x)
    if(na_num==length(x)){
        return(NA)
    }
    if(na_num >max(tab)){
        return(NA)
    }else{
        return(as.numeric(names(which.max(table(temp2)))))
    }
}
weight_average_simmery <- function(x,pos){
    len=length(x)-1
    weight = c(seq(1,pos-1)/(len*(len+1)/4),seq(pos-1,1)/(len*(len+1)/4))
    rebalance = sum(weight*(!is.na(x[-pos])))
    weight = weight[(!is.na(x[-pos]))]/rebalance
    return(round(sum(x[-pos][(!is.na(x[-pos]))]*weight),0))
}

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.1.9000     ✔ purrr   0.3.2     
✔ tibble  2.1.3          ✔ dplyr   0.8.3     
✔ tidyr   1.0.0          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.4.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 


Attaching package: ‘xts’


The following objects are masked from ‘package:data.table’:

    first, last


The following objects are masked from ‘package:dplyr’:

[1] "building_metadata.csv" "sample_submission.csv" "test.csv"             
[4] "train.csv"             "weather_test.csv"      "weather_train.csv"

In [2]:
## load data
xtrain <- fread(paste(path,'train.csv',sep=''))
weather_train = fread(paste(path,'weather_train.csv',sep=''))
weather_test = fread(paste(path,'weather_test.csv',sep=''))
building_metadata = fread(paste(path,'building_metadata.csv',sep=''))

In [3]:
# # the column we need to know what to drop 
# xtrain$tokeep <- 0

# ## detect the strings of zeroes ----

# # list of unique buildings 
# building_list <- unique(xtrain$building_id)


# for (bb in building_list){
#     index0 <- which(xtrain$building_id == bb)
#     x0 <- xtrain[index0,]
#     x0[, ':='(rmx = rollmean(meter_reading, k = 50, partial = TRUE, fill = 0, align = 'right')),
#            by= list(meter)]
#     idx <- which(x0$rmx < .5)
#     xtrain$tokeep[index0[-idx]] <- 1
#     print(paste(bb, ': ', round(1-length(idx)/nrow(x0),2), ' kept', sep = ''))
# }

# xtr <- xtrain[xtrain$tokeep == 1,]
# xtr$tokeep <- NULL

In [4]:
## CORRECT RANGE FOR PHYSICAL DATA
#Wind speed can't be less than zero
# weather_train$wind_speed[weather_train$wind_speed<0] = 0

# weather_test$wind_speed[weather_test$wind_speed<0] = 0

# #Wind direction should be in range [0,360] --> correct by compass
# weather_train$wind_direction[weather_train$wind_direction>360] = weather_train$wind_direction[weather_train$wind_direction>360]-360
# weather_test$wind_direction[weather_test$wind_direction>360] = weather_test$wind_direction[weather_test$wind_direction>360]-360

# weather_train$wind_direction[weather_train$wind_direction<0] = 360 - weather_train$wind_direction[weather_train$wind_direction<0]
# weather_test$wind_direction[weather_test$wind_direction<0] = 360 - weather_test$wind_direction[weather_test$wind_direction<0]

weather_train$wind_direction[weather_train$wind_direction==360]=0
weather_test$wind_direction[weather_test$wind_direction==360]=0

In [5]:
siter = sort(unique(weather_train$site_id))
weather_train$timestamp = as.POSIXct(weather_train$timestamp)
weather_test$timestamp = as.POSIXct(weather_test$timestamp)

time_train_filter = max(weather_train$timestamp)

#cange weather_train
original_time = seq(min(weather_train$timestamp),max(weather_train$timestamp),by='1 hour')

for(site in siter){
    cat('Number of Missing for site',site,': ',length(original_time)-sum(weather_train$site_id==site),'\n')
}

cat('----------------------------------\n')
original_time = seq(min(weather_test$timestamp),max(weather_test$timestamp),by='1 hour')

for(site in siter){
    cat('Number of Missing for site',site,': ',length(original_time)-sum(weather_test$site_id==site),'\n')
}

#return to char for left join
weather_train$timestamp = as.character(weather_train$timestamp)
weather_test$timestamp = as.character(weather_test$timestamp)




Number of Missing for site 0 :  0 
Number of Missing for site 1 :  21 
Number of Missing for site 2 :  1 
Number of Missing for site 3 :  4 
Number of Missing for site 4 :  1 
Number of Missing for site 5 :  29 
Number of Missing for site 6 :  2 
Number of Missing for site 7 :  170 
Number of Missing for site 8 :  0 
Number of Missing for site 9 :  4 
Number of Missing for site 10 :  2 
Number of Missing for site 11 :  170 
Number of Missing for site 12 :  29 
Number of Missing for site 13 :  1 
Number of Missing for site 14 :  7 
Number of Missing for site 15 :  330 
----------------------------------
Number of Missing for site 0 :  0 
Number of Missing for site 1 :  233 
Number of Missing for site 2 :  0 
Number of Missing for site 3 :  3 
Number of Missing for site 4 :  4 
Number of Missing for site 5 :  279 
Number of Missing for site 6 :  16 
Number of Missing for site 7 :  947 
Number of Missing for site 8 :  0 
Number of Missing for site 9 :  39 
Number of Missing for site 10 : 

In [6]:
weather_train$timestamp = as.POSIXct(weather_train$timestamp)
weather_test$timestamp = as.POSIXct(weather_test$timestamp)
time_seq = seq(min(weather_train$timestamp),max(weather_test$timestamp)+10*60*60,by='1 hour')
site_id = rep(siter,each=length(time_seq))

weather_train = data.frame(weather_train,to_corr=1)
weather_test = data.frame(weather_test,to_corr=1)

weather_all = left_join(data.frame(site_id = site_id,timestamp = time_seq),rbind.data.frame(weather_train,weather_test),
                       by=c('site_id'='site_id','timestamp'='timestamp'))


In [7]:
dum_vec<- weather_all$to_corr
weather_all <- weather_all[,-ncol(weather_all)]

In [8]:
# weather_train = data.frame(weather_train)
# for(site in siter){
#     pos_all_site = which(weather_train$site_id==site)
#     for(car in 1:ncol(weather_train)){
#         if(mean(is.na(weather_train[pos_all_site,car]))>0){
#             cat('Number of Missing for site',site,', col ',names(weather_train)[car],' : ',mean(is.na(weather_train[pos_all_site,car])),'\n')
#         }
#     }
# }


In [9]:
for(site in siter){
    pos_all_site = which(weather_all$site_id==site)
    for(car in 1:ncol(weather_all)){
        if(mean(is.na(weather_all[pos_all_site,car]))>0.1){
            cat('Number of Missing for site',site,', col ',names(weather_all)[car],' : ',mean(is.na(weather_all[pos_all_site,car])),'\n')
        }
    }
}


Number of Missing for site 0 , col  cloud_coverage  :  0.4325834 
Number of Missing for site 1 , col  cloud_coverage  :  0.7980163 
Number of Missing for site 1 , col  precip_depth_1_hr  :  1 
Number of Missing for site 2 , col  cloud_coverage  :  0.2968002 
Number of Missing for site 3 , col  cloud_coverage  :  0.4374097 
Number of Missing for site 4 , col  cloud_coverage  :  0.4664817 
Number of Missing for site 5 , col  cloud_coverage  :  0.6723797 
Number of Missing for site 5 , col  precip_depth_1_hr  :  1 
Number of Missing for site 5 , col  sea_level_pressure  :  1 
Number of Missing for site 6 , col  cloud_coverage  :  0.3605685 
Number of Missing for site 7 , col  cloud_coverage  :  1 
Number of Missing for site 7 , col  precip_depth_1_hr  :  0.9039295 
Number of Missing for site 8 , col  cloud_coverage  :  0.4325834 
Number of Missing for site 9 , col  cloud_coverage  :  0.426237 
Number of Missing for site 9 , col  wind_direction  :  0.3044767 
Number of Missing for site 10 

In [10]:
#CORRECT MISSING FOR ALL EXCEPT CLOUD COVERAGE AND PRECIPIT WHICH HAS NATURALLY MISSING VALUES.

to_corr = which(names(weather_all)%in%c('air_temperature','dew_temperature','sea_level_pressure'))
siter = sort(unique(weather_all$site_id))
for(site in siter){
    pos_all_site = which(weather_all$site_id==site)
    for(car in to_corr){
        cat('Number of Missing for site',site,', col ',names(weather_all)[car],' : ',round(mean(is.na(weather_all[pos_all_site,car]))*100,2),'\n')
        if(mean(is.na(weather_all[pos_all_site,car]))<.4){
            weather_all[pos_all_site,car] = round(na.interp(ts(weather_all[pos_all_site,car],frequency=24)),1)
        }
    }
}

Number of Missing for site 0 , col  air_temperature  :  0.05 
Number of Missing for site 0 , col  dew_temperature  :  0.05 
Number of Missing for site 0 , col  sea_level_pressure  :  1.54 
Number of Missing for site 1 , col  air_temperature  :  1.09 
Number of Missing for site 1 , col  dew_temperature  :  1.09 
Number of Missing for site 1 , col  sea_level_pressure  :  1.45 
Number of Missing for site 2 , col  air_temperature  :  0.05 
Number of Missing for site 2 , col  dew_temperature  :  0.05 
Number of Missing for site 2 , col  sea_level_pressure  :  0.33 
Number of Missing for site 3 , col  air_temperature  :  0.09 
Number of Missing for site 3 , col  dew_temperature  :  0.1 
Number of Missing for site 3 , col  sea_level_pressure  :  2.09 
Number of Missing for site 4 , col  air_temperature  :  0.06 
Number of Missing for site 4 , col  dew_temperature  :  0.08 
Number of Missing for site 4 , col  sea_level_pressure  :  0.79 
Number of Missing for site 5 , col  air_temperature  :  

In [11]:
#EXPONENTIAL BACK AND FORWARD APROXIMATION 

to_corr = which(names(weather_all)%in%c('wind_direction','wind_speed'))
siter = sort(unique(weather_all$site_id))
window = 10
for(site in siter){
    pos_site = which(weather_all$site_id==site)
    for(car in to_corr){
        pos = which(is.na(weather_all[pos_site,car]))
        for(i in pos){
            if(i>window & i<(max(pos_site)-window)){
                micro_series = weather_all[pos_site[seq(i-window,i+window)],car]
                weather_all[pos_site[i],car]=weight_average_simmery(micro_series,window+1)
            }
        }
    }
}

#MEAN BY HOUR AND MONTH FOR REMAINING MISSING VALUES

month_t = month(weather_all$timestamp)
day_t = day(weather_all$timestamp) 
for(site in siter){
    pos_site = which(weather_all$site_id==site)
    temp = weather_all[pos_site,]
    month_t_temp = month_t[pos_site]
    day_t_temp = day_t[pos_site]
    for(car in to_corr){
        pos = which(is.na(temp[,car]))
        if(length(pos)>0){
            for(i in pos){
                temp2 = na.omit(temp[month_t_temp==month_t_temp[i] & day_t_temp == day_t_temp[i],car])
                weather_all[pos_site[i],car]=round(mean(temp2))
            }
        }
    }
}




In [12]:
### CLOUD AS MOST FREQUENT VALUES WITH ALSO MISSING


to_corr = which(names(weather_all)%in%c('cloud_coverage','precip_depth_1_hr'))
siter = sort(unique(weather_all$site_id))

month_t = month(weather_all$timestamp)
day_t = day(weather_all$timestamp) 
# res <- c()
for(site in siter){
    pos_site = which(weather_all$site_id==site)
    temp = weather_all[pos_site,]
    month_t_temp = month_t[pos_site]
    day_t_temp = day_t[pos_site]
    dum_temp = dum_vec[pos_site]
    for(car in to_corr){
        pos = which(is.na(dum_temp))
        if(length(pos)>0){
            for(i in pos){
                temp2 = temp[month_t_temp==month_t_temp[i] & day_t_temp == day_t_temp[i] & (!is.na(dum_temp)),car]
#                 res<-c(res,freq_na_calc(temp2))
                weather_all[pos_site[i],car]=freq_na_calc(temp2)
            }
        }
    }
}


In [13]:
#check outlier
# to_corr = which(names(weather_all)%in%c('sea_level_pressure','wind_speed')) #'air_temperature','dew_temperature','wind_direction',
# siter = sort(unique(weather_all$site_id))
# for(site in siter){
#     pos_all_site = which(weather_all$site_id==site)
#     for(car in to_corr){
#         if(mean(is.na(weather_all[pos_all_site,car]))<.4){
#             out = tsoutliers(ts(weather_all[pos_all_site,car],frequency=24))
#             if(length(out$index)>0){
#                 cat('Number of Outlier for site',site,', col ',names(weather_all)[car],' : ',length(out$index),'\n')
#             }
#         }
#     }
# }

In [14]:
weather_all$timestamp = as.character(weather_all$timestamp)

weather_train = data.table(weather_all[weather_all$timestamp<=time_train_filter,])
weather_test = data.table(weather_all[weather_all$timestamp>time_train_filter,])

rm(weather_all)

In [15]:
building_metadata[,c('age_built'):=list(2019-year_built)][,year_built:=NULL]
#,'new_century_built' ,as.numeric(year_built>2000)  trunc(logger


In [16]:
#MEAN INFORMATION FOR BUILD BY SITE ID AND OTHER

# building_metadata[,c('square_feat_mean_site','age_built_mean_site',
#                      'new_century_built_mean_site'):=list(trunc(mean(logger(square_feet),na.rm=T),4),
#                                                           trunc(mean(logger(age_built),na.rm=T),4),
#                                                           trunc(mean(logger(new_century_built),na.rm=T),4)),by=site_id]
# building_metadata[,c('square_feat_mean_primary','age_built_mean_primary',
#                      'new_century_built_mean_primary'):=list(trunc(mean(logger(square_feet),na.rm=T),4),
#                                                              trunc(mean(logger(age_built),na.rm=T),4),
#                                                              trunc(mean(logger(new_century_built),na.rm=T),4)),by=primary_use]
# building_metadata[,c('square_feat_mean_primary_site','age_built_mean_primary_site',
#                      'new_century_built_mean_primary_site'):=list(trunc(mean(logger(square_feet),na.rm=T),4),
#                                                                   trunc(mean(logger(age_built),na.rm=T),4),
#                                                                   trunc(mean(logger(new_century_built),na.rm=T),4)),
#                   by=list(site_id,primary_use)]

building_metadata$square_feet = logger(building_metadata$square_feet)
building_metadata$primary_use = as.numeric(as.factor(building_metadata$primary_use))


In [17]:
# a=weather_train[,list(air_temperature=mean(is.na(air_temperature)),dew_temperature=mean(is.na(air_temperature)),
#                     cloud_coverage=mean(is.na(cloud_coverage)),
#                     precip_depth_1_hr=mean(is.na(precip_depth_1_hr)),
#                     sea_level_pressure=mean(is.na(sea_level_pressure)),
#                     wind_direction=mean(is.na(wind_direction)),wind_speed=mean(is.na(wind_speed))),by=site_id]

In [18]:
# b = weather_test[,list(air_temperature=mean(is.na(air_temperature)),dew_temperature=mean(is.na(air_temperature)),
#                     cloud_coverage=mean(is.na(cloud_coverage)),
#                     precip_depth_1_hr=mean(is.na(precip_depth_1_hr)),
#                     sea_level_pressure=mean(is.na(sea_level_pressure)),
#                     wind_direction=mean(is.na(wind_direction)),wind_speed=mean(is.na(wind_speed))),by=site_id]

In [19]:
#SOME SITE HAS ALL RELEVATION MISSING --> CREATE DUMMY

# weather_train[,c('has_cloud','has_rain','has_sea','precipit_long') := list(
#                                         ifelse(weather_train$site_id%in%b$site_id[a$cloud_coverage==1],1,0),
#                                         ifelse(weather_train$site_id%in%b$site_id[a$precip_depth_1_hr==1],1,0),
#                                         ifelse(weather_train$site_id%in%b$site_id[a$sea_level_pressure==1],1,0),
#                                         ifelse(weather_train$precip_depth_1_hr==(-1),1,0))]
# weather_test[,c('has_cloud','has_rain','has_sea','precipit_long') := list(
#                                         ifelse(weather_test$site_id%in%b$site_id[b$cloud_coverage==1],1,0),
#                                         ifelse(weather_test$site_id%in%b$site_id[b$precip_depth_1_hr==1],1,0),
#                                         ifelse(weather_test$site_id%in%b$site_id[b$sea_level_pressure==1],1,0),
#                                         ifelse(weather_test$precip_depth_1_hr==(-1),1,0))]

In [20]:
#EVERY MISSING IN TO CORR ARE CORRECTED WITH WEIGHTED MEAN BY DECAYING FACTOR TO GIVE BETTER IMPORTANCE TO RECENT OBSERVATION

# to_corr = which(names(weather_train)%in%c('air_temperature','dew_temperature','sea_level_pressure','wind_direction','wind_speed'))
# siter = unique(weather_train$site_id)
# weather_train = data.frame(weather_train)
# for(site in siter){
#     pos_site = which(weather_train$site_id==site)
#     for(car in to_corr){
#         pos = which(is.na(weather_train[pos_site,car]))
#         for(i in pos){
#             if(i>5){
#                 micro_series = na.omit(weather_train[pos_site[seq(i-5,i)],car])
#                 if(length(micro_series)>0){
#                     weather_train[pos_site[i],car]=weight_average(micro_series)#mean(weather_train[pos_site[seq(i-3,i)],car],na.rm=T)
#                 }
#             }
#         }
#     }
# }


In [21]:
#EVERY MISSING IN TO CORR ARE CORRECTED WITH WEIGHTED MEAN BY DECAYING FACTOR TO GIVE BETTER IMPORTANCE TO RECENT OBSERVATION

# to_corr = which(names(weather_test)%in%c('air_temperature','dew_temperature','sea_level_pressure','wind_direction','wind_speed'))
# siter = unique(weather_test$site_id)
# weather_test = data.frame(weather_test)
# for(site in siter){
#     pos_site = which(weather_test$site_id==site)
#     for(car in to_corr){
#         pos = which(is.na(weather_test[pos_site,car]))
#         for(i in pos){
#             if(i>5){
#                 micro_series = na.omit(weather_test[pos_site[seq(i-5,i)],car])
#                 if(length(micro_series)>0){
#                     weather_test[pos_site[i],car]=weight_average(micro_series)#mean(weather_train[pos_site[seq(i-3,i)],car],na.rm=T)
#                 }
#             }
            
#         }
#     }
# }
# weather_train = weather_train[-which(is.na(weather_train$wind_direction)),]

In [22]:
#WIND DIRECTION COMPASS
beaufort = c(0, 0.3, 1.6, 3.4, 5.5, 8, 10.8, 13.9, 17.2, 20.8, 24.5, 28.5, 33)

weather_train$wind_compass = round(weather_train$wind_direction/22.5 +.5)%%16
weather_test$wind_compass = round(weather_test$wind_direction/22.5 +.5)%%16

#BEAUFORT SCALE
weather_train$beaufort_scale = as.numeric(cut(weather_train$wind_speed,breaks = beaufort))
weather_test$beaufort_scale = as.numeric(cut(weather_test$wind_speed,breaks = beaufort))


#RELATIVE HUMIDITY
weather_train$relative_humidity =  relative_temp(weather_train$dew_temperature,weather_train$air_temperature)
weather_test$relative_humidity = relative_temp(weather_test$dew_temperature,weather_test$air_temperature)

#HEAT INDEX
weather_train$heat_index =  heat_index(weather_train$air_temperature,weather_train$relative_humidity)
weather_test$heat_index = heat_index(weather_test$air_temperature,weather_test$relative_humidity)

weather_train$wind_chill =  wind_chill(weather_train$wind_speed,weather_train$air_temperature)
weather_test$wind_chill = wind_chill(weather_test$wind_speed,weather_test$air_temperature)

weather_train$wind_load =  weather_train$sea_level_pressure*(weather_train$wind_speed^2)
weather_test$wind_load = weather_test$sea_level_pressure*(weather_test$wind_speed^2)

weather_train$temp_diff =  weather_train$dew_temperature-weather_train$air_temperature
weather_test$temp_diff = weather_test$dew_temperature-weather_test$air_temperature

weather_train$feel = feel_like(weather_train$wind_speed,weather_train$air_temperature,weather_train$relative_humidity)
weather_test$feel = feel_like(weather_test$wind_speed,weather_test$air_temperature,weather_test$relative_humidity)



In [23]:
wind_chill =function(v,t){
    t = farenhait_converter(t)
    v = wind_converter(v)
    return(ifelse(v>=3 & t<=50,35.74 + (0.6215 * t) - 35.75 * (v^0.16) + 0.4275 * t * v^0.16,NA))
}


In [24]:
train_pos = nrow(weather_train)
to_corr = which(names(weather_train)%in%c('air_temperature','dew_temperature','sea_level_pressure',
                                         'relative_humidity','wind_speed','heat_index','wind_chill','wind_load','temp_diff'))
To_erase = c()
n=1
weather_train = data.frame(weather_train)
weather_test = data.frame(weather_test)

if(to_diff){
    for(site in unique(weather_train$site_id)){
        To_erase = c(To_erase,which(weather_train$site_id==site)[1:n])
        for(car in to_corr){
            train_temp = weather_train[weather_train$site_id==site,car]
            train_temp_pos = length(train_temp) 
            test_temp = weather_test[weather_test$site_id==site,car]
            temp = c(train_temp,test_temp)
            calc_temp = differ(temp,n=n)#trunc(logger(temp),4)
            weather_train[weather_train$site_id==site,car] = calc_temp[1:train_temp_pos]
            weather_test[weather_test$site_id==site,car] = calc_temp[(train_temp_pos+1):length(temp)]
        }
    }
    rm(list=c('train_temp','test_temp','temp','calc_temp'))

}
if(to_log){
    for(site in unique(weather_train$site_id)){
        To_erase = c(To_erase,which(weather_train$site_id==site)[1:n])
        for(car in to_corr){
            train_temp = weather_train[weather_train$site_id==site,car]
            train_temp_pos = length(train_temp) 
            test_temp = weather_test[weather_test$site_id==site,car]
            temp = c(train_temp,test_temp)
            calc_temp = logger(temp)#
            weather_train[weather_train$site_id==site,car] = calc_temp[1:train_temp_pos]
            weather_test[weather_test$site_id==site,car] = calc_temp[(train_temp_pos+1):length(temp)]
        }
    }
    rm(list=c('train_temp','test_temp','temp','calc_temp'))

}
# weather_train = weather_train[-To_erase,]

In [25]:
if(tolag==TRUE){
    to_lag = c('air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed',
               'wind_compass','beaufort_scale','relative_humidity','heat_index','wind_chill','wind_load','temp_diff')
    train_pos = nrow(weather_train)
    To_erase = c()
    n=1

    weather_train[,paste(to_lag,'lag_24',sep='_')]=0
    weather_train[,paste(to_lag,'lag_48',sep='_')]=0

    weather_test[,paste(to_lag,'lag_24',sep='_')]=0
    weather_test[,paste(to_lag,'lag_48',sep='_')]=0

    for(site in unique(weather_train$site_id)){
        To_erase = c(To_erase,which(weather_train$site_id==site)[1:49])
        for(car in to_lag){
            train_temp = weather_train[weather_train$site_id==site,car]
            train_temp_pos = length(train_temp) 
            test_temp = weather_test[weather_test$site_id==site,car]
            temp = c(train_temp,test_temp)
            calc_temp_24 = data.table::shift(temp,24)
            calc_temp_48 = data.table::shift(temp,48)

            weather_train[weather_train$site_id==site,paste(car,'lag_24',sep='_')] = calc_temp_24[1:train_temp_pos]
            weather_test[weather_test$site_id==site,paste(car,'lag_24',sep='_')] = calc_temp_24[(train_temp_pos+1):length(temp)]

            weather_train[weather_train$site_id==site,paste(car,'lag_48',sep='_')] = calc_temp_48[1:train_temp_pos]
            weather_test[weather_test$site_id==site,paste(car,'lag_48',sep='_')] = calc_temp_48[(train_temp_pos+1):length(temp)]

        }
    }
    weather_train <- weather_train[-To_erase,]
}

In [26]:
train = fread(paste(path,'train.csv',sep=''))
test = fread(paste(path,'test.csv',sep=''))


In [27]:
if(leak==TRUE){
    
#     leak_df_ucf = fread(paste(ucf_root,'site0.csv',sep=''))
#     leak_df_ucf$meter_reading = leak_df_ucf$meter_reading_scraped
#     leak_df_ucf[,c('meter_reading_original','meter_reading_scraped'):=list(NULL,NULL)]
#     leak_df_ucf = leak_df_ucf[-which(is.na(leak_df_ucf$meter_reading)),]
#     leak_df_ucf[leak_df_ucf$meter_reading < 0, 'meter_reading'] = 0
#     leak_df_ucf = leak_df_ucf[year(leak_df_ucf$timestamp)> 2016,]
#     leak_df_ucf = leak_df_ucf[,c('timestamp','building_id','meter','meter_reading')]

#     leak_df_ucl = fread(paste(ucl_root,'site1.csv',sep=''))
#     leak_df_ucl$meter_reading = leak_df_ucl$meter_reading_scraped
#     leak_df_ucl[,c('meter_reading_scraped'):=list(NULL)]
#     leak_df_ucl = leak_df_ucl[-which(is.na(leak_df_ucl$meter_reading)),]
#     leak_df_ucl[leak_df_ucl$meter_reading < 0, 'meter_reading'] = 0
#     leak_df_ucl = leak_df_ucl[year(leak_df_ucl$timestamp)> 2016,]
#     leak_df_ucl = leak_df_ucl[,c('timestamp','building_id','meter','meter_reading')]

#     leak_df_asu = fread(paste(asu_root,'site2.csv',sep=''))
#     leak_df_asu = leak_df_asu[-which(is.na(leak_df_asu$meter_reading)),]
#     leak_df_asu[leak_df_asu$meter_reading < 0, 'meter_reading'] = 0
#     leak_df_asu = leak_df_asu[year(leak_df_asu$timestamp) > 2016,]
#     leak_df_asu = leak_df_asu[,c('timestamp','building_id','meter','meter_reading')]


#     leak_df_cor = fread(paste(cor_root,'site15.csv',sep=''))
#     leak_df_cor[leak_df_cor$meter_reading < 0, 'meter_reading'] = 0
#     leak_df_cor = leak_df_cor[year(leak_df_cor$timestamp) > 2016,]
#     leak_df_cor = leak_df_cor[,c('timestamp','building_id','meter','meter_reading')]

#     leak_df = rbind.data.frame(leak_df_ucf,leak_df_ucl,leak_df_asu,leak_df_cor)
#     leak_df = unique(leak_df)
#     leak_df = leak_df[order(timestamp,building_id,meter)]
    library(feather)
    leak_df = read_feather('../input/ashrae-leak-data-station/leak.feather')
    leak_df$timestamp = as.character(leak_df$timestamp)
    leak_df = leak_df[,c('building_id','meter','timestamp','meter_reading')]
    train = rbind.data.frame(train,leak_df)
    train = unique(train)
    train = train[order(timestamp,building_id,meter)]

}


In [28]:
train[,diff_meter:=paste(sort(unique(meter)),collapse=''),by=building_id]
test[,diff_meter:=paste(sort(unique(meter)),collapse=''),by=building_id]

train$diff_meter = as.integer(as.factor(train$diff_meter))
test$diff_meter = as.integer(as.factor(test$diff_meter))

train[,building_meter:=paste(building_id,meter,sep='')]
test[,building_meter:=paste(building_id,meter,sep='')]

train$building_meter = as.integer(as.factor(train$building_meter))
test$building_meter = as.integer(as.factor(test$building_meter))
test[,row_id:=NULL]


In [29]:
train_now = nrow(train)
all =rbind.data.frame(train[,c('building_id','meter','diff_meter','building_meter')],
                      test[,c('building_id','meter','diff_meter','building_meter')])
all[,c('building_id_count'):=list(.N/nrow(all)),by =building_id]
all[,c('meter_count'):=list(.N/nrow(all)),by =meter]
all[,c('diff_meter_count'):=list(.N/nrow(all)),by =diff_meter]
all[,c('building_meter_count'):=list(.N/nrow(all)),by =building_meter]

In [30]:
train[,c('building_id_count','meter_count','diff_meter_count','building_meter_count')] = all[1:train_now,c('building_id_count','meter_count','diff_meter_count','building_meter_count')]
test[,c('building_id_count','meter_count','diff_meter_count','building_meter_count')] = all[(train_now+1):nrow(all),c('building_id_count','meter_count','diff_meter_count','building_meter_count')]

In [31]:
rm(all)

In [32]:
h5createFile("data.h5")
h5write(train, "data.h5","train")
h5write(test, "data.h5","test")
h5write(building_metadata, "data.h5","building_metadata")
h5write(weather_train, "data.h5","weather_train")
h5write(weather_test, "data.h5","weather_test")


[1] TRUE